In [1]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import praw
import time
import datetime
from datetime import datetime, timedelta

In [2]:
client = MongoClient('localhost', 27017)
db = client.db
headlines = db.headlines
threads = db.threads
analysis = db.analysis

In [3]:
df = pd.read_csv('companylist.csv')

In [4]:
secretfile = open('secret.txt')
reddit = praw.Reddit(client_id='-wpcPIbA7bhlpw', client_secret=secretfile.read(), user_agent='sentiment')

In [5]:
wantedTags = ['Daily Discussion', 'DD', 'Technicals', 'Fundamentals', 'Discussion', 'YOLO', 'Stocks']
subreddit = reddit.subreddit('wallstreetbets')
postlist = []

for post in subreddit.hot(limit = 1000):
    if datetime.today().weekday == 0:
        if len(postlist) < 100:
            postTime = post.created
            postDate = datetime.utcfromtimestamp(postTime)
            
            currentTime = datetime.utcnow()
            
            postDelta = currentTime - postDate

            if postDelta.days < 3 and post.link_flair_text in wantedTags:
                postlist.append(post)
        else:
            break
    
    else:
        if len(postlist) < 100:
            postTime = post.created
            postDate = datetime.utcfromtimestamp(postTime)

            currentTime = datetime.utcnow()

            postDelta = currentTime - postDate
            postDelta = str(postDelta)

            if 'day' not in postDelta and post.link_flair_text in wantedTags:
                postlist.append(post)
        else:
            break

In [6]:
postDf = pd.DataFrame(columns = ['Date','Title','Body'])

In [7]:
dates = '{}/{}/{}'.format(currentTime.month,currentTime.day,currentTime.year)
titles = [post.title for post in postlist]
bodies = [post.selftext for post in postlist]
tags = [post.link_flair_text for post in postlist]

In [8]:
postDf['Title'] = titles
postDf['Body'] = bodies
postDf['Date'] = dates
postDf['Tag'] = tags

In [9]:
stocklist = df['Symbol']

In [10]:
stocklist
stocklistcopy = ['$'+stocklist]
stocklist = stocklist.append(stocklistcopy).tolist()

In [31]:
def findStocks(string):
    stockset = set(string.split()).intersection(stocklist)
    return stockset

In [41]:
postDf['set1'] = postDf['Body'].apply(findStocksBody)
postDf['set2'] = postDf['Title'].apply(findStocksBody)
postDf['Stocks'] = postDf.apply(lambda x: x['set1'].union(x['set2']), axis = 1)
postDf.drop(columns = ['set1','set2'])